# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe69c4219a0>
├── 'a' --> tensor([[ 1.6354, -0.3869, -1.1288],
│                   [-0.0901, -0.6869, -1.0016]])
└── 'x' --> <FastTreeValue 0x7fe69c421fd0>
    └── 'c' --> tensor([[-1.7645,  0.9949,  1.4610,  0.6240],
                        [-0.4780, -0.7030, -2.7138, -1.9123],
                        [-1.0011, -1.5927, -1.3506, -1.1088]])

In [4]:
t.a

tensor([[ 1.6354, -0.3869, -1.1288],
        [-0.0901, -0.6869, -1.0016]])

In [5]:
%timeit t.a

63.7 ns ± 0.071 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe69c4219a0>
├── 'a' --> tensor([[-0.7453, -0.7756,  1.2442],
│                   [-0.3212,  1.1157, -0.7227]])
└── 'x' --> <FastTreeValue 0x7fe69c421fd0>
    └── 'c' --> tensor([[-1.7645,  0.9949,  1.4610,  0.6240],
                        [-0.4780, -0.7030, -2.7138, -1.9123],
                        [-1.0011, -1.5927, -1.3506, -1.1088]])

In [7]:
%timeit t.a = new_value

73.4 ns ± 0.0283 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.6354, -0.3869, -1.1288],
               [-0.0901, -0.6869, -1.0016]]),
    x: Batch(
           c: tensor([[-1.7645,  0.9949,  1.4610,  0.6240],
                      [-0.4780, -0.7030, -2.7138, -1.9123],
                      [-1.0011, -1.5927, -1.3506, -1.1088]]),
       ),
)

In [10]:
b.a

tensor([[ 1.6354, -0.3869, -1.1288],
        [-0.0901, -0.6869, -1.0016]])

In [11]:
%timeit b.a

64.3 ns ± 0.14 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1884,  0.4098, -0.9540],
               [-0.4476,  1.7211,  0.4970]]),
    x: Batch(
           c: tensor([[-1.7645,  0.9949,  1.4610,  0.6240],
                      [-0.4780, -0.7030, -2.7138, -1.9123],
                      [-1.0011, -1.5927, -1.3506, -1.1088]]),
       ),
)

In [13]:
%timeit b.a = new_value

517 ns ± 0.35 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

837 ns ± 7.42 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 23.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 323 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe5d29b8a90>
├── 'a' --> tensor([[[ 1.6354, -0.3869, -1.1288],
│                    [-0.0901, -0.6869, -1.0016]],
│           
│                   [[ 1.6354, -0.3869, -1.1288],
│                    [-0.0901, -0.6869, -1.0016]],
│           
│                   [[ 1.6354, -0.3869, -1.1288],
│                    [-0.0901, -0.6869, -1.0016]],
│           
│                   [[ 1.6354, -0.3869, -1.1288],
│                    [-0.0901, -0.6869, -1.0016]],
│           
│                   [[ 1.6354, -0.3869, -1.1288],
│                    [-0.0901, -0.6869, -1.0016]],
│           
│                   [[ 1.6354, -0.3869, -1.1288],
│                    [-0.0901, -0.6869, -1.0016]],
│           
│                   [[ 1.6354, -0.3869, -1.1288],
│                    [-0.0901, -0.6869, -1.0016]],
│           
│                   [[ 1.6354, -0.3869, -1.1288],
│                    [-0.0901, -0.6869, -1.0016]]])
└── 'x' --> <FastTreeValue 0x7fe5d96693a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

34.1 µs ± 69.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe69c4534f0>
├── 'a' --> tensor([[ 1.6354, -0.3869, -1.1288],
│                   [-0.0901, -0.6869, -1.0016],
│                   [ 1.6354, -0.3869, -1.1288],
│                   [-0.0901, -0.6869, -1.0016],
│                   [ 1.6354, -0.3869, -1.1288],
│                   [-0.0901, -0.6869, -1.0016],
│                   [ 1.6354, -0.3869, -1.1288],
│                   [-0.0901, -0.6869, -1.0016],
│                   [ 1.6354, -0.3869, -1.1288],
│                   [-0.0901, -0.6869, -1.0016],
│                   [ 1.6354, -0.3869, -1.1288],
│                   [-0.0901, -0.6869, -1.0016],
│                   [ 1.6354, -0.3869, -1.1288],
│                   [-0.0901, -0.6869, -1.0016],
│                   [ 1.6354, -0.3869, -1.1288],
│                   [-0.0901, -0.6869, -1.0016]])
└── 'x' --> <FastTreeValue 0x7fe6a14aa0d0>
    └── 'c' --> tensor([[-1.7645,  0.9949,  1.4610,  0.6240],
                        [-0.4780, -0.7030, -2.7138, -1.9123],
                 

In [23]:
%timeit t_cat(trees)

32 µs ± 71 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65 µs ± 50.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.6354, -0.3869, -1.1288],
                [-0.0901, -0.6869, -1.0016]],
       
               [[ 1.6354, -0.3869, -1.1288],
                [-0.0901, -0.6869, -1.0016]],
       
               [[ 1.6354, -0.3869, -1.1288],
                [-0.0901, -0.6869, -1.0016]],
       
               [[ 1.6354, -0.3869, -1.1288],
                [-0.0901, -0.6869, -1.0016]],
       
               [[ 1.6354, -0.3869, -1.1288],
                [-0.0901, -0.6869, -1.0016]],
       
               [[ 1.6354, -0.3869, -1.1288],
                [-0.0901, -0.6869, -1.0016]],
       
               [[ 1.6354, -0.3869, -1.1288],
                [-0.0901, -0.6869, -1.0016]],
       
               [[ 1.6354, -0.3869, -1.1288],
                [-0.0901, -0.6869, -1.0016]]]),
    x: Batch(
           c: tensor([[[-1.7645,  0.9949,  1.4610,  0.6240],
                       [-0.4780, -0.7030, -2.7138, -1.9123],
                       [-1.0011, -1.5927, -1.3506, -1.1088]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.6354, -0.3869, -1.1288],
               [-0.0901, -0.6869, -1.0016],
               [ 1.6354, -0.3869, -1.1288],
               [-0.0901, -0.6869, -1.0016],
               [ 1.6354, -0.3869, -1.1288],
               [-0.0901, -0.6869, -1.0016],
               [ 1.6354, -0.3869, -1.1288],
               [-0.0901, -0.6869, -1.0016],
               [ 1.6354, -0.3869, -1.1288],
               [-0.0901, -0.6869, -1.0016],
               [ 1.6354, -0.3869, -1.1288],
               [-0.0901, -0.6869, -1.0016],
               [ 1.6354, -0.3869, -1.1288],
               [-0.0901, -0.6869, -1.0016],
               [ 1.6354, -0.3869, -1.1288],
               [-0.0901, -0.6869, -1.0016]]),
    x: Batch(
           c: tensor([[-1.7645,  0.9949,  1.4610,  0.6240],
                      [-0.4780, -0.7030, -2.7138, -1.9123],
                      [-1.0011, -1.5927, -1.3506, -1.1088],
                      [-1.7645,  0.9949,  1.4610,  0.6240],
                      [-0.4780, -

In [28]:
%timeit Batch.cat(batches)

149 µs ± 280 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

335 µs ± 989 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
